In [1]:
# load lib
import tabulate
import numpy as np
from tqdm import tqdm
from det3.ops import read_pkl


In [2]:
# global variables
g_task="task1"
g_domain="domain1"
if g_task == "task3":
    g_statistic_dict = {
        "joint_training": [f"July14-expcarlarmore-jointtraining_cv{cv}_{g_task}_{g_domain}" for cv in range(5)],
        "lwf": [f"July14-expcarlarmore-lwf_cv{cv}_{g_task}_{g_domain}" for cv in range(5)],
        "fine_tuning": [f"July14-expcarlarmore-finetuning_cv{cv}_{g_task}_{g_domain}" for cv in range(3)]
    }
else:
    g_statistic_dict = {
        "joint_training": [f"July14-expcarlarmore-jointtraining_cv{cv}_{g_task}_{g_domain}" for cv in range(5)],
        "lwf": [f"July14-expcarlarmore-lwf_cv{cv}_{g_task}_{g_domain}" for cv in range(5)],
        "fine_tuning": [f"July14-expcarlarmore-finetuning_cv{cv}_{g_task}_{g_domain}" for cv in range(5)]
    }
g_valid_classes = None
g_mAP_weights = None
g_mAPs = {}
g_cls2metric= {
    "Car": "3d@0.50",
    "Pedestrian": "3d@0.25",
    "Cyclist": "3d@0.25"
}

In [3]:
# define functions
def get_valid_classes(task:str, domain:str):
    if domain == "domain3":
        l_domain = ["Car", "Pedestrian", "Cyclist"]
    elif domain == "domain2":
        l_domain = ["Car", "Pedestrian"]
    elif domain == "domain1":
        l_domain = ["Car"]
    else:
        raise NotImplementedError
    if task == "task3":
        l_task = ["Car", "Pedestrian", "Cyclist"]
    elif task == "task2":
        l_task = ["Car", "Pedestrian"]
    elif task == "task1":
        l_task = ["Car"]
    else:
        raise NotImplementedError
    return [itm for itm in l_domain if itm in l_task]
    

def tag2pkl(tag, task:str, domain:str, cls2metric:dict):
    '''
    return result pkl according to tag
    '''
    if task == "task3":
        path = f"/usr/app/incdet3/logs/July14-expcarlamore-eval/{tag}/test_results.pkl"
    elif domain == "domain3" and task != "task1":
        path = f"/usr/app/incdet3/logs/July14-expcarlamore-eval/{tag}/test_results.pkl"
    elif task == "task2" and domain=="domain2":
        tag_ = tag.split("-")[2].split("_")[:2]
        tag_ = "_".join(tag_) if not "lwf" in tag else f"lwf_distloss_{tag_[1]}"
        tag_ = f"{tag_}-afteril-targetdataset-last"
        path = f"/usr/app/incdet3/logs/July07-expcarla-il/eval_log/{tag_}/test_results.pkl"
    elif task == "task2" and domain=="domain1":
        tag_ = tag.split("-")[2].split("_")[:2]
        tag_ = "_".join(tag_) if not "lwf" in tag else f"lwf_distloss_{tag_[1]}"
        tag_ = f"{tag_}-afteril-sourcedataset-last"
        path = f"/usr/app/incdet3/logs/July07-expcarla-il/eval_log/{tag_}/test_results.pkl"
    elif domain == "domain3" and task == "task1":
        path = "/usr/app/incdet3/logs/July14-expcarlamore-eval/July14-expcarlarmore-task1_domain3/test_results.pkl"
    elif task == "task1" and domain == "domain2":
        path = "/usr/app/incdet3/logs/July07-expcarla-il/eval_log/beforeil-targetdataset/test_results.pkl"
    elif task == "task1" and domain == "domain1":
        path = "/usr/app/incdet3/logs/July07-expcarla-il/eval_log/beforeil-sourcedataset/test_results.pkl"
    data = read_pkl(path)
    data = data['detail']['eval.carla']['carla']
    res = {}
    for cls, v in data.items():
        res[cls] = v[cls2metric[cls]]
    return res

def domain2infopkl(domain:str):
    '''
    return the test info pkl according to domain.
    '''
    if domain == "domain3":
        path = "/usr/app/data/CARLA-TOWN01CARPEDCYC/CARLA_infos_test.pkl"
    elif domain == "domain2":
        path = "/usr/app/data/CARLA-TOWN02CARPED/CARLA_infos_test.pkl"
    elif domain == "domain1":
        path = "/usr/app/data/CARLA-TOWN01CAR/CARLA_infos_test.pkl"
    else:
        raise NotImplementedError
    return read_pkl(path)
    
def compute_mAP(eval_data:dict, mAP_weights:dict):
    '''
    return the mAP:
    {
    "Car": float,
    "Pedestrian": float,
    "Cyclist": float
    }
    '''
    res = {}
    for cls, aps in eval_data.items():
        if cls not in [k for k in mAP_weights.keys()]:
            continue
        res[cls] = sum([ap * weight / sum(mAP_weights[cls])
                    for ap, weight in zip(aps, mAP_weights[cls])])
    return res

In [4]:
# compute easy, mod, hard weight for each class of the domain
# mAP_weights = {
#     "Car": [num_of_easy, num_of_mod, num_of_hard],
#     ...
# }
g_valid_classes = get_valid_classes(g_task, g_domain)
info_pkl = domain2infopkl(g_domain)
mAP_weights = {k: [0, 0, 0] for k in g_valid_classes}
for itm in tqdm(info_pkl):
    label = itm["label"]
    for obj in label.data:
        if obj.type not in g_valid_classes:
            continue
        dist = np.sqrt(obj.x **2 + obj.y **2)
        if dist <= 20:
            mAP_weights[obj.type][0] += 1
        if dist <= 30:
            mAP_weights[obj.type][1] += 1
        if dist <= 50:
            mAP_weights[obj.type][2] += 1
print(mAP_weights)
g_mAP_weights = mAP_weights

100%|██████████| 1100/1100 [00:00<00:00, 7104.90it/s]

{'Car': [1357, 2657, 5797]}


In [5]:
# load each evaluation results and compute its map
for _, v in g_statistic_dict.items():
    for tag in v:
        eval_data = tag2pkl(tag, g_task, g_domain, g_cls2metric)
        mAP = compute_mAP(eval_data, g_mAP_weights)
        g_mAPs[tag] = mAP
# for k, v in g_mAPs.items():
#     print(k)
#     for kk, vv in v.items():
#         print(f"  {kk}: {vv:.1f}")

In [6]:
# generate a table according to the g_statistic_dict
header = ["Case"] + g_valid_classes
content = []
for statistic_tag, tags in g_statistic_dict.items():
    ctt = [statistic_tag]
    for cls in g_valid_classes:
        data = [g_mAPs[tag][cls] for tag in tags]
        ctt += [f"{np.mean(data):.1f}+-{np.std(data):.1f}"]
    content.append(ctt)
print(tabulate.tabulate(content, headers=header, tablefmt="pipe"))

| Case           | Car       |
|:---------------|:----------|
| joint_training | 63.4+-0.0 |
| lwf            | 63.4+-0.0 |
| fine_tuning    | 63.4+-0.0 |
